## Batch processing with Argo Worfklows

In this notebook we will dive into how you can run batch processing with Argo Workflows and Seldon Core.

Dependencies:

* Seldon core installed as per the docs with an ingress
* Minio running in your cluster to use as local (s3) object storage
* Argo Workfklows installed in cluster (and argo CLI for commands)


## Setup

### Install Seldon Core
Use the notebook to [set-up Seldon Core with Ambassador or Istio Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

### Set up Minio in your cluster
Use the notebook to [set-up Minio in your cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/minio_setup.html).

### Copy the Minio Secret to namespace

We need to re-use the minio secret for the batch job, so this can be done by just copying the minio secret created in the `minio-system`

The command below just copies the secred with th e name "minio" from the minio-system namespace to the default namespace.

In [ ]:
!kubectl get secret minio -n minio-system -o json | jq '{apiVersion,data,kind,metadata,type} | .metadata |= {"annotations", "name"}' | kubectl apply -n default -f -

## Create some input for our model

We will create a file that will contain the inputs that will be sent to our model

In [38]:
mkdir -p assets/

In [2]:
with open("assets/input-data.txt", "w") as f:
    for i in range(10000):
        f.write('[[1, 2, 3, 4]]\n')

### Check the contents of the file

In [3]:
!wc -l assets/input-data.txt
!head assets/input-data.txt

10000 assets/input-data.txt
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]
[[1, 2, 3, 4]]


### Upload the file to our minio

In [5]:
!mc mb minio-seldon/data
!mc cp assets/input-data.txt minio-seldon/data/

Bucket created successfully `minio-seldon/data`.
...-data.txt:  146.48 KiB / 146.48 KiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 2.03 MiB/s 0s

### Create Argo Workflow

In order to create our argo workflow we have made it simple so you can leverage the power of the helm charts.

Before we dive into the contents of the full helm chart, let's first give it a try with some of the settings.

We will run a batch job that will set up a Seldon Deployment with 10 replicas and 100 batch client workers to send requests.

In [29]:
!helm template seldon-batch-workflow helm-charts/seldon-batch-workflow/ \
    --set workflow.name=seldon-batch-process \
    --set seldonDeployment.name=sklearn \
    --set seldonDeployment.replicas=10 \
    --set batchWorker.workers=100 \
    --set batchWorker.payloadType=ndarray \
    --set batchWorker.dataType=data \
    | argo submit -

Name:                seldon-batch-process
Namespace:           default
ServiceAccount:      default
Status:              Pending
Created:             Mon Jun 08 11:57:56 +0100 (1 second ago)


In [7]:
!argo list

NAME                   STATUS    AGE   DURATION   PRIORITY
seldon-batch-process   Running   3s    3s         0


In [30]:
!argo get seldon-batch-process

Name:                seldon-batch-process
Namespace:           default
ServiceAccount:      default
Status:              Succeeded
Created:             Mon Jun 08 11:57:56 +0100 (1 minute ago)
Started:             Mon Jun 08 11:57:57 +0100 (1 minute ago)
Finished:            Mon Jun 08 11:59:33 +0100 (10 seconds ago)
Duration:            1 minute 36 seconds

STEP                                                             PODNAME                          DURATION  MESSAGE
 ✔ seldon-batch-process (seldon-batch-process)                                                              
 ├---✔ create-seldon-resource (create-seldon-resource-template)  seldon-batch-process-3626514072  1s        
 ├---✔ wait-seldon-resource (wait-seldon-resource-template)      seldon-batch-process-2052519094  32s       
 ├---✔ download-object-store (download-object-store-template)    seldon-batch-process-1257652469  3s        
 ├---✔ process-batch-inputs (process-batch-inputs-template)      seldon-batch-process-2

In [31]:
!argo logs -w seldon-batch-process 

create-seldon-resource:	time="2020-06-08T10:57:58Z" level=info msg="Starting Workflow Executor" version=v2.8.0-rc4+8f69617.dirty
create-seldon-resource:	time="2020-06-08T10:57:58Z" level=info msg="Creating a docker executor"
create-seldon-resource:	time="2020-06-08T10:57:58Z" level=info msg="Executor (version: v2.8.0-rc4+8f69617.dirty, build_date: 2020-05-12T15:17:15Z) initialized (pod: default/seldon-batch-process-3626514072) with template:\n{\"name\":\"create-seldon-resource-template\",\"arguments\":{},\"inputs\":{},\"outputs\":{},\"metadata\":{},\"resource\":{\"action\":\"create\",\"manifest\":\"apiVersion: machinelearning.seldon.io/v1\\nkind: SeldonDeployment\\nmetadata:\\n  name: \\\"sklearn\\\"\\n  namespace: default\\n  ownerReferences:\\n  - apiVersion: argoproj.io/v1alpha1\\n    blockOwnerDeletion: true\\n    kind: Workflow\\n    name: \\\"seldon-batch-process\\\"\\n    uid: \\\"15014702-d5a7-4dc6-a219-373b7013b744\\\"\\nspec:\\n  name: \\\"sklearn\\\"\\n  predictors:\\n    - 

## Check output in object store

We can now visualise the output that we obtained in the object store.

First we can check that the file is present:

In [32]:
import json
wf_arr = !argo get seldon-batch-process -o json
wf = json.loads("".join(wf_arr))
WF_ID = wf["metadata"]["uid"]
print(f"Workflow ID is {WF_ID}")

Workflow ID is 15014702-d5a7-4dc6-a219-373b7013b744


In [34]:
!mc ls minio-seldon/data/output-data-"$WF_ID".txt

[2020-06-08 11:59:31 BST]  2.7MiB output-data-15014702-d5a7-4dc6-a219-373b7013b744.txt


Now we can output the contents of the file created using the `mc head` command.

In [36]:
!mc cp minio-seldon/data/output-data-"$WF_ID".txt assets/output-data.txt
!head assets/output-data.txt

...3b744.txt:  2.75 MiB / 2.75 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 24.68 MiB/s 0s{"data": {"names": ["t:0", "t:1", "t:2"], "ndarray": [[0.0006985194531162841, 0.003668039039435755, 0.9956334415074478]]}, "meta": {"tags": {"tags": {"batch_id": "03403156-a977-11ea-88f0-ea5c4746b555", "batch_index": 2.0, "batch_instance_id": "0340bbe4-a977-11ea-9ca4-ea5c4746b555"}}}}
{"data": {"names": ["t:0", "t:1", "t:2"], "ndarray": [[0.0006985194531162841, 0.003668039039435755, 0.9956334415074478]]}, "meta": {"tags": {"tags": {"batch_id": "03403156-a977-11ea-88f0-ea5c4746b555", "batch_index": 6.0, "batch_instance_id": "03411da0-a977-11ea-9ca4-ea5c4746b555"}}}}
{"data": {"names": ["t:0", "t:1", "t:2"], "ndarray": [[0.0006985194531162841, 0.003668039039435755, 0.9956334415074478]]}, "meta": {"tags": {"tags": {"batch_id": "03403156-a977-11ea-88f0-ea5c4746b555", "batch_index": 9.0, "batch_instance_id": "03415234-a977-11ea-9ca4-ea5c4746b555"}}}}
{"data": {"names": ["t:0", "t:1", "t:2"], "ndarray": [[0.00069

In [37]:
!argo delete seldon-batch-process

Workflow 'seldon-batch-process' deleted
